In [88]:
import yaml
import sys
import os
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [3]:
config = "../config/p-type.yml"
with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

In [74]:
class MLP(tf.keras.Model):

    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(4, activation="linear")
        self.dropout = tf.keras.layers.Dropout(0.25)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        if training:
            x = self.dropout(x, training=training)
        x = self.dense2(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.dense3(x)
        return x

In [75]:
if not os.path.isfile(os.path.join(conf['data_path'], "cached.parquet")):
    df = pd.concat([
        pd.read_parquet(x) for x in tqdm.tqdm(glob.glob(os.path.join(conf['data_path'], "*.parquet")))
    ])
    df.to_parquet(os.path.join(conf['data_path'], "cached.parquet"))
else:
    df = pd.read_parquet(os.path.join(conf['data_path'], "cached.parquet"))

### Split and preprocess the data
df['day'] = df['datetime'].apply(lambda x: str(x).split(' ')[0])
df["id"] = range(df.shape[0])

In [76]:
# Need the same test_data for all trained models (data and model ensembles)
n_splits = 1
flat_seed = 1000
data_seed = 0
gsp = GroupShuffleSplit(n_splits=1,  random_state = flat_seed, train_size=0.9)
splits = list(gsp.split(df, groups = df["day"]))
train_index, test_index = splits[0]
train_data, test_data = df.iloc[train_index].copy(), df.iloc[test_index].copy() 

# Make N train-valid splits using day as grouping variable
gsp = GroupShuffleSplit(n_splits=n_splits,  random_state = flat_seed, train_size=0.885)
splits = list(gsp.split(train_data, groups = train_data["day"]))
train_index, valid_index = splits[data_seed]
train_data, valid_data = train_data.iloc[train_index].copy(), train_data.iloc[valid_index] .copy() 

In [110]:
features = conf['tempvars'] + conf['tempdewvars'] + conf['ugrdvars'] + conf['vgrdvars']
outputs = conf['outputvars']
num_classes = len(outputs)

scaler_x = StandardScaler()
x_train = scaler_x.fit_transform(train_data[features])
x_valid = scaler_x.transform(valid_data[features])
x_test = scaler_x.transform(test_data[features])
y_train = tf.keras.utils.to_categorical(np.argmax(train_data[outputs].to_numpy(), 1), num_classes)
y_valid = tf.keras.utils.to_categorical(np.argmax(valid_data[outputs].to_numpy(), 1), num_classes)
y_test = tf.keras.utils.to_categorical(np.argmax(test_data[outputs].to_numpy(), 1), num_classes)

In [118]:
lgamma = tf.math.lgamma
digamma = tf.math.digamma

epochs = [1]

def KL(alpha, num_classes=4):
    one = K.constant(np.ones((1,num_classes)),dtype=tf.float32)
    S = K.sum(alpha,axis=1,keepdims=True)  
    A1 = K.sum(lgamma(alpha), axis=1, keepdims=True)
    A2 = K.sum(lgamma(one), axis=1, keepdims=True)
    A3 = lgamma(K.sum(one, axis=1, keepdims=True))
    A4 = K.sum((alpha - one)*(digamma(alpha)-digamma(S)), axis=1, keepdims=True)
    kl = lgamma(S) - A1 + A2 - A3 + A4
          
    return kl

def loss_func(y_true, output):
    y_evidence = K.relu(output)
    alpha = y_evidence+1
    S = K.sum(alpha,axis=1,keepdims=True)
    p = alpha / S  

    err = K.sum(K.pow((y_true-p),2),axis=1,keepdims=True)
    var = K.sum(alpha*(S-alpha)/(S*S*(S+1)),axis=1,keepdims=True)
    
    l = K.sum(err + var, axis=1, keepdims=True)
    l = K.sum(l)
    
    kl =  K.minimum(1.0, epochs[0]/50) * K.sum(KL((1-y_true)*(alpha)+y_true))
    return l + kl

In [177]:
del mlp
K.clear_session()
gc.collect()

23200

In [178]:
mlp = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(units = 128, activation = 'relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units = 128, activation = 'relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units = 4, activation = 'linear'),
    ]
)

In [179]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-3
)

In [180]:
# compile model
mlp.compile(
    optimizer = optimizer, 
    loss = loss_func,
    metrics = ['accuracy'],
    #run_eagerly=True
)

In [181]:
# create early stopping callback
callback1 = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy', 
    mode = 'max',
    patience = 15,
    restore_best_weights = True
)

# create reduce LR callback
callback2 = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_accuracy', 
    factor = 0.1, 
    patience = 3, 
    verbose = 0,
    mode = 'max',
    min_lr = 1e-7
)

In [182]:
# fit model to training data
history = mlp.fit(
    x = x_train, 
    y = y_train,
    validation_data = (x_valid, y_valid),
    batch_size = conf["trainer"]["batch_size"],
    epochs = 1000,
    callbacks = [callback1, callback2],
    verbose = 2,
    shuffle = True
)

Epoch 1/1000
176/176 - 1s - loss: 549.6086 - accuracy: 0.8416 - val_loss: 489.8992 - val_accuracy: 0.8459 - lr: 0.0010 - 1s/epoch - 8ms/step
Epoch 2/1000
176/176 - 1s - loss: 406.2060 - accuracy: 0.8804 - val_loss: 459.3554 - val_accuracy: 0.8586 - lr: 0.0010 - 522ms/epoch - 3ms/step
Epoch 3/1000
176/176 - 1s - loss: 387.1797 - accuracy: 0.8862 - val_loss: 453.8924 - val_accuracy: 0.8613 - lr: 0.0010 - 506ms/epoch - 3ms/step
Epoch 4/1000
176/176 - 1s - loss: 377.1010 - accuracy: 0.8895 - val_loss: 444.6345 - val_accuracy: 0.8642 - lr: 0.0010 - 503ms/epoch - 3ms/step
Epoch 5/1000
176/176 - 1s - loss: 370.1025 - accuracy: 0.8918 - val_loss: 444.8278 - val_accuracy: 0.8628 - lr: 0.0010 - 506ms/epoch - 3ms/step
Epoch 6/1000
176/176 - 1s - loss: 366.3386 - accuracy: 0.8928 - val_loss: 431.2420 - val_accuracy: 0.8685 - lr: 0.0010 - 508ms/epoch - 3ms/step
Epoch 7/1000
176/176 - 1s - loss: 362.2213 - accuracy: 0.8940 - val_loss: 434.3902 - val_accuracy: 0.8685 - lr: 0.0010 - 506ms/epoch - 3ms/

In [186]:
def calc_prob_uncertinty(outputs):
    evidence = K.relu(outputs)
    alpha = evidence + 1
    u = 4 / K.sum(alpha, axis=1, keepdims=True)
    prob = alpha / K.sum(alpha, axis=1, keepdims=True)
    return prob.numpy(), u.numpy()

In [187]:
preds = mlp.predict(x_test)

1261/1261 [==============================] - 1s 865us/step


In [188]:
prob, u = calc_prob_uncertinty(preds)

In [189]:
prob, u

(array([[0.97840726, 0.00719758, 0.00719758, 0.00719758],
        [0.9840467 , 0.00531777, 0.00531777, 0.00531777],
        [0.97950155, 0.00683282, 0.00683282, 0.00683282],
        ...,
        [0.953632  , 0.015456  , 0.015456  , 0.015456  ],
        [0.9398522 , 0.02004927, 0.02004927, 0.02004927],
        [0.9856658 , 0.00477808, 0.00477808, 0.00477808]], dtype=float32),
 array([[0.0287903 ],
        [0.02127108],
        [0.02733126],
        ...,
        [0.06182401],
        [0.08019707],
        [0.0191123 ]], dtype=float32))